In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import ipdb

In [2]:
class Node(object):
    def __init__(self, typ, val):
        self.type = typ
        self.value = val
        self.Str = '%s(%s)'%(self.type, self.value)
        self.left = None
        self.right = None
        self.leftStr = '_'
        self.rightStr = '_'
    def __str__(self):
        return '%s) -> [%s, %s]'%(self.Str, self.leftStr, self.rightStr)
    def setLeft(self, L):
        self.left = L
        self.leftStr = '%s(%s)'%(self.left.type, self.left.value)
    def setRight(self, R):
        self.right = R
        self.rightStr = '%s(%s)'%(self.right.type, self.right.value)

In [3]:
def BuildTree(currNode, currDepth, maxDepth, nodes, nodeType, nodeLens):
    #ipdb.set_trace()
    # exit if too deep or at a leaf
    if (currDepth > maxDepth) or (currNode.type != 'ops'):
        return currNode
    # hit max depth, so ensure only consts or feats selected
    if currDepth == maxDepth:
        nt = [t for t in nodeTypes if t != 'ops']
        nodeTypeL = nt[np.random.randint(3)]
        nodeTypeR = nt[np.random.randint(3)]
    else:
        nodeTypeL = nodeTypes[np.random.randint(3)]
        nodeTypeR = nodeTypes[np.random.randint(3)]
        
    # randomly generate the left node    
    nodeValuL = nodes[nodeTypeL][np.random.randint(nodeLens[nodeTypeL])]
    currNode.setLeft(BuildTree(Node(nodeTypeL, nodeValuL), currDepth+1, maxDepth, nodes, nodeType, nodeLens))

    # randomly generate the right node    
    nodeValuR = nodes[nodeTypeR][np.random.randint(nodeLens[nodeTypeR])]
    currNode.setRight(BuildTree(Node(nodeTypeR, nodeValuR), currDepth+1, maxDepth, nodes, nodeType, nodeLens))
    
    return currNode

In [14]:
def RecPrintTree(currNode, resDict, currKey):
    resDict[currKey] += currNode.Str+'|'
    if (currNode.left is None) & (currNode.right is None):
        return resDict
    
    if currNode.left is not None:
        resDict = RecPrintTree(currNode.left, resDict, currKey+1)
    if currNode.right is not None:
        resDict = RecPrintTree(currNode.right, resDict, currKey+1)
    return resDict

def PrintTree(topNode, maxDepth):
    # populate the tree view dict
    resDict = dict.fromkeys(list(range(maxDepth)), '')
    resDict = RecPrintTree(topNode, resDict, 0)
    # unpopulate it now
    for key in list(resDict.keys()):
        if resDict[key] == '':
            resDict.pop(key)
    
    return resDict

In [5]:
ops = ['+', '-', '*', '\\', '^', 'max', 'min']
feats = ['x%d'%i for i in range(5)]
consts = list(range(1, 11))

nodes = {'ops':ops, 'feats':feats, 'consts':consts}
nodeLens = {'ops':len(ops), 'feats':len(feats), 'consts':len(consts)}
nodeTypes = list(nodes.keys())

In [19]:
''' randomly generate some trees '''
# set seed
nw = dt.datetime.now()
randSeed = nw.hour*10000 + nw.minute*100 + nw.second
print('Random seed = %d'%randSeed)
np.random.seed(randSeed)

# set the depth
maxDepth = 10
depth = np.random.randint(low=1, high=maxDepth+1)

# build the tree, starting from the top node
treeCnt = 10
trees = [None]*treeCnt
for t in range(treeCnt):
    nodeType = nodeTypes[np.random.randint(3)]
    nodeValu = nodes[nodeType][np.random.randint(nodeLens[nodeType])]
    trees[t] = BuildTree(Node(nodeType, nodeValu), 0, 3, nodes, nodeType, nodeLens)
    #print(trees[t])
    print(PrintTree(trees[t], maxDepth))

Random seed = 131320
{0: 'ops(max)|', 1: 'feats(x4)|feats(x2)|'}
{0: 'ops(^)|', 1: 'ops(*)|consts(4)|', 2: 'feats(x0)|feats(x2)|'}
{0: 'consts(1)|'}
{0: 'consts(1)|'}
{0: 'ops(max)|', 1: 'ops(-)|ops(max)|', 2: 'ops(min)|consts(3)|consts(4)|consts(4)|', 3: 'consts(3)|ops(max)|', 4: 'consts(10)|feats(x2)|'}
{0: 'consts(3)|'}
{0: 'feats(x0)|'}
{0: 'feats(x2)|'}
{0: 'feats(x4)|'}
{0: 'feats(x3)|'}
